In [ ]:
import ngsolve as ngs
from ngsolve.webgui import Draw
import netgen.occ as occ

from dream.incompressible import IncompressibleSolver, flowfields, Force
import dream.bla as bla

# Channel flow
H = 2.0

face = occ.WorkPlane().RectangleC(3, H).Face()
for edge, name in zip(face.edges, ["bottom", "right", "top", "left"]):
    edge.name = name
face.edges[1].Identify(face.edges[3], "periodic", occ.IdentificationType.PERIODIC)
face.name = "channel"
geo = occ.OCCGeometry(face, dim=2)
mesh = ngs.Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)

In [ ]:
# Set up the solver configuration
cfg = IncompressibleSolver(mesh)
cfg.time = "stationary"
cfg.reynolds_number = 1
cfg.dynamic_viscosity = "constant"

cfg.fem = "taylor-hood"
cfg.fem.order = 2
cfg.fem.scheme = "direct"

cfg.nonlinear_solver.damping_factor = 1
cfg.nonlinear_solver.max_iterations = 100

In [ ]:
PRESSURE_DROP = 2.0

cfg.bcs.clear()
cfg.dcs.clear()
cfg.bcs['left|right'] = "periodic"
cfg.bcs['top|bottom'] = "wall"
cfg.dcs['channel'] = Force((PRESSURE_DROP, 0))

In [ ]:
# Initialize the finite element spaces, trial and test functions, gridfunctions, and boundary conditions
cfg.initialize()

Uh = cfg.get_solution_fields('velocity', default_fields=False)
cfg.io.draw(Uh)

# Solve solution
cfg.solve()

cfg.io.undraw()

In [ ]:
# Change the viscosity model to power-law fluid
cfg.dynamic_viscosity = "powerlaw"
cfg.dynamic_viscosity.powerlaw_exponent = 1.6


# Exact solution for the power-law fluid
r = cfg.dynamic_viscosity.powerlaw_exponent
dP = PRESSURE_DROP

ue = flowfields()
ue.u = ((r-1)/r * (dP)**(1/(r-1)) * (H/2)**(r/(r-1)) * (1 - (2*bla.abs(ngs.y)/H)**(r/(r-1))), 0)
ue.p = 0

Draw(ue.u, mesh, "exact_velocity")

In [ ]:
cfg.fem.initialize_symbolic_forms()



cfg.io.draw(Uh)

# Solve the system
cfg.solve()

cfg.io.undraw()

In [ ]:
# Change the viscosity model to power-law fluid
cfg.dynamic_viscosity = "bingham"
cfg.dynamic_viscosity.yield_stress = 0.5
cfg.dynamic_viscosity.kappa = 1e-2


cfg.fem.initialize_symbolic_forms()


cfg.io.draw(Uh)

# Solve the system
cfg.solve()

cfg.io.undraw()